# Geomatching

This notebook shows how to use the `Geomatcher` object in `constructive_geometries`. Download the [data here](https://files.brightway.dev/faces.gpkg).

## Consistent global topology

The foundation for the `geomatcher` is a consistent global topology, built using the [Constructive Geometries](https://bitbucket.org/cmutel/constructive-geometries) repository, and provided as a Python library here. The base data is from [Natural Earth](http://www.naturalearthdata.com/), though there has been a number of fixes and added locations.

A [topology](https://postgis.net/docs/Topology.html) is useful because it comes with guarantees of consistency, as each edge is only stored once. This allows us to do GIS operations using set algebra with the ids of topological faces. Here is an part of the world, as provided in `Constructive Geometries`:

<img src="japan.png">

As you can see, each polygon is a face, no matter how big or small it is.

## Retrieving facial ids

As a first try, you can retrieve the faces associated with any given location:

In [ ]:
from constructive_geometries import *
geomatcher = Geomatcher()
list(geomatcher['JP'])[:10]

Geospatial definitions are namespaced, except for countries. Countries are therefore defined by their ISO two-letter codes, but other data should be referenced by a tuple of its namespace and identifier:

In [ ]:
list(geomatcher[('ecoinvent', 'NAFTA')])[:10]

You can also set a default namespace. The default is `"ecoinvent"`. So, the IMAGE regions are loaded by default, and must be retrieved explicitly, while ecoinvent regions will be searched automatically, unless the default namespace is changed.

In [ ]:
list(geomatcher['NAFTA'])[:10]

Finally, by default `geomatcher` will search for country names, using [country converter](https://github.com/konstantinstadler/country_converter).

In [ ]:
'Japan' in geomatcher.topology

In [ ]:
list(geomatcher['Japan'])[:10]

## GIS operations: intersection, contained, within

Geomatcher allows you to do quick GIS calculations.

In [ ]:
geomatcher.intersects("US")

`contained` gets all locations that are *completely within* this location, whereas `within` gets all locations that *completely contain* this location.

In [ ]:
geomatcher.contained("US")

In [ ]:
geomatcher.within("US")

For all three operations, you can exclude the input variable:

In [ ]:
geomatcher.within("US", include_self=False)

You can also change the sorting order, with is biggest first by default.

In [ ]:
geomatcher.within("US", include_self=False, biggest_first=False)

Finally, you can ask for an list where none of the regions overlap:

In [ ]:
geomatcher.intersects("US", biggest_first=False, exclusive=True)

### Rest of World and `resolved_row`

By default, the location 'Rest of World' or 'RoW' does not have a fixed spatial definition - it is defined dynamically by whatever is left over some regions are defined. So `RoW("France")` would be the rest of the world, aside from France. As such, 'RoW' is never returned by the GIS functions in `geomatcher`, unless it is defined.

You can use the `resolved_row` context manager to define the rest of the world temporarily:

In [ ]:
assert 'RoW' not in geomatcher

with resolved_row(["FR", "GB"], geomatcher) as g:
    assert 'RoW' in g
    
assert 'RoW' not in geomatcher

In this case, the 'US' is not in the 'RoW':

In [ ]:
with resolved_row(["NAFTA"], geomatcher) as g:
    print(geomatcher.within("US"))

When the 'RoW' is a valid response, it is treated the same as any other location - it has a set of faces, so will be sorted in the correct order.

In [ ]:
with resolved_row(["FR"], geomatcher) as g:
    print(geomatcher.within("US"))

In [ ]:
with resolved_row(["FR"], geomatcher) as g:
    print(geomatcher.within("US", biggest_first=False))

## Splitting faces

Say we wanted to split the island of Honshu and develop a separate inventory for Tokyo. From our graphic above, we know that Honshu has face number 6247. So, we can split this face into two new ones - one of which we will consider Tolyo, and the other the "Rest of Honshu".

In [ ]:
first, second = geomatcher.split_face(6247)
first, second

In [ ]:
geomatcher.topology[("example", "Rest of Honshu")] = set([first])
geomatcher.topology[("example", "Tokyo")] = set([second])

In [ ]:
geomatcher.contained("JP")

`split_face` also supports the arguments `number` (number of new faces to create), and `ids` (integers values for the new ids to create). If `ids` is passed, `number` is ignored.

## Adding new topologies

You can also add new topologies to support custom spatial systems. This is how the IMAGE regions are added in Wurst:

    geomatcher.add_definitions(IMAGE_TOPOLOGY, "IMAGE")
    
New topologies can be either relative (default) or not. Relative topologies are defined by reference to regions already in the topology:

    {"Russia Region": [
        "AM",
        "AZ",
        "GE",
        "RU"
    ]}
    
Non-relative topologies must be defined by a set of integer ids.

    {
        'A': {1, 2, 3},
        'B': {2, 3, 4},
    }
    
Regions added by `add_definitions` will be namespaced with the second argument passed to the function.

In [ ]:
topoz = {"Black Sea": [
    "RO",
    "TR",
    "UA",
    "GE",
    "BG",
    "RU",
]}
geomatcher.add_definitions(topoz, "just added")

In [ ]:
geomatcher.contained(("just added", "Black Sea"))